In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indic-bert/indic-bert/config.json
/kaggle/input/indic-bert/indic-bert/spiece.vocab
/kaggle/input/indic-bert/indic-bert/spiece.model
/kaggle/input/indic-bert/indic-bert/README.md
/kaggle/input/indic-bert/indic-bert/tf_model.ckpt.meta
/kaggle/input/indic-bert/indic-bert/tf_model.ckpt.data-00000-of-00001
/kaggle/input/indic-bert/indic-bert/pytorch_model.bin
/kaggle/input/indic-bert/indic-bert/.gitattributes
/kaggle/input/indic-bert/indic-bert/tf_model.ckpt.index
/kaggle/input/indic-bert/indic-bert/.git/config
/kaggle/input/indic-bert/indic-bert/.git/packed-refs
/kaggle/input/indic-bert/indic-bert/.git/HEAD
/kaggle/input/indic-bert/indic-bert/.git/index
/kaggle/input/indic-bert/indic-bert/.git/description
/kaggle/input/indic-bert/indic-bert/.git/info/exclude
/kaggle/input/indic-bert/indic-bert/.git/refs/heads/main
/kaggle/input/indic-bert/indic-bert/.git/refs/remotes/origin/HEAD
/kaggle/input/indic-bert/indic-bert/.git/hooks/pre-merge-commit.sample
/kaggle/input/indic-bert/in

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)



2026-01-04 14:32:49.477556: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767537169.684864      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767537169.738978      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767537170.245101      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767537170.245139      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767537170.245142      55 computation_placer.cc:177] computation placer alr

Using device: cuda


In [3]:
import pandas as pd

# Replace with your Tamil dataset paths
train = pd.read_csv("/kaggle/input/tamil-dataset/tamil_hope_first_train.csv", sep="\t", header=None)
dev   = pd.read_csv("/kaggle/input/tamil-dataset/tamil_hope_first_dev.csv", sep="\t", header=None)
test  = pd.read_csv("/kaggle/input/tamil-dataset/tamil_hope_first_test.csv", sep="\t", header=None)

train.columns = ["raw"]
dev.columns = ["raw"]
test.columns = ["raw"]


In [4]:
def clean_split_tamil(df):
    df = df.copy()
    # split by ";" or whatever separator your dataset uses
    parts = df["raw"].str.split(";", expand=True)
    
    df["text"] = parts[0]
    df["label"] = parts[1]

    # drop rows without labels
    df = df.dropna(subset=["label"])
    
    # remove 'not-Tamil' rows
    df = df[df["label"] != "not-Tamil"]

    # keep only text + label
    df = df[["text", "label"]]
    return df

train = clean_split_tamil(train)
dev   = clean_split_tamil(dev)
test  = clean_split_tamil(test)


In [5]:
import re

def clean_text_tamil(t):
    t = str(t).lower()
    t = re.sub(r"http\S+|www\S+", "", t)       # remove URLs
    t = re.sub(r"@\w+", "", t)                 # remove usernames
    t = re.sub(r"\s+", " ", t).strip()         # remove extra spaces
    return t

train["text"] = train["text"].apply(clean_text_tamil)
dev["text"]   = dev["text"].apply(clean_text_tamil)
test["text"]  = test["text"].apply(clean_text_tamil)


In [6]:
train

,text,label
1,i also don't have tiktok hello and allnbut i'm...,Hope_speech
2,thalaivare..neengale inum one plus mobile vach...,Non_hope_speech
3,annee varanda thondai.. corona virus affect pa...,Hope_speech
4,5views but 18likes,Non_hope_speech
5,china phone vakathiga inu evanga ellam tiktok ...,Non_hope_speech
...,...,...
16155,7pm correcta erukum mg bro,Hope_speech
16156,intha karutha mudija varaikum neengalum ellark...,Non_hope_speech
16157,and neenga adhiyavasiyam nu soldra apps like t...,Non_hope_speech
16158,daii sekram mater ku vada,Hope_speech


In [7]:
label_map = {
    "Hope_speech": 1,
    "Non_hope_speech": 0
}

train["label"] = train["label"].map(label_map)
dev["label"]   = dev["label"].map(label_map)
test["label"]  = test["label"].map(label_map)

In [8]:
train

,text,label
1,i also don't have tiktok hello and allnbut i'm...,1.0
2,thalaivare..neengale inum one plus mobile vach...,0.0
3,annee varanda thondai.. corona virus affect pa...,1.0
4,5views but 18likes,0.0
5,china phone vakathiga inu evanga ellam tiktok ...,0.0
...,...,...
16155,7pm correcta erukum mg bro,1.0
16156,intha karutha mudija varaikum neengalum ellark...,0.0
16157,and neenga adhiyavasiyam nu soldra apps like t...,0.0
16158,daii sekram mater ku vada,1.0


In [13]:
train_texts = train["text"].tolist()
dev_texts   = dev["text"].tolist()
test_texts  = test["text"].tolist()

In [15]:

from sentence_transformers import SentenceTransformer

mpnet_model = SentenceTransformer('all-mpnet-base-v2', device=device)

train_mpnet = mpnet_model.encode(train_texts, batch_size=32, show_progress_bar=True)
dev_mpnet   = mpnet_model.encode(dev_texts, batch_size=32)
test_mpnet  = mpnet_model.encode(test_texts, batch_size=32)


Batches:   0%|          | 0/444 [00:00<?, ?it/s]

In [18]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
indic_model_path = "/kaggle/input/indic-bert/indic-bert"  # update path

indic_tokenizer = AutoTokenizer.from_pretrained(indic_model_path)
indic_model = AutoModel.from_pretrained(indic_model_path).to(device)
indic_model.eval()

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * mask, dim=1) / torch.clamp(mask.sum(dim=1), min=1e-9)

def extract_indicbert_embeddings(texts, batch_size=32):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        encoded = indic_tokenizer(batch_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
        encoded = {k: v.to(device) for k, v in encoded.items()}
        with torch.no_grad():
            output = indic_model(**encoded)
        embeddings = mean_pooling(output, encoded["attention_mask"])
        embeddings = F.normalize(embeddings, p=2, dim=1)
        all_embeddings.append(embeddings.cpu())
    return torch.cat(all_embeddings).numpy()

# Extract embeddings
train_indic_emb = extract_indicbert_embeddings(train_texts)
dev_indic_emb   = extract_indicbert_embeddings(dev_texts)
test_indic_emb  = extract_indicbert_embeddings(test_texts)


In [20]:
#CNN(AGNOSTIC)
import torch
import torch.nn as nn
import torch.nn.functional as F

class HopeSpeechCNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()

        self.fc1 = nn.Linear(input_dim, input_dim)
        self.dropout = nn.Dropout(0.25)

        self.conv1 = nn.Conv1d(1, 64, kernel_size=5)
        self.pool1 = nn.MaxPool1d(4)

        self.conv2 = nn.Conv1d(64, 64, kernel_size=5)
        self.pool2 = nn.MaxPool1d(4)

        self.conv3 = nn.Conv1d(64, 64, kernel_size=5)
        self.pool3 = nn.MaxPool1d(4)

        # Dynamically compute output size
        self._dummy_forward(input_dim)

        self.fc_out = nn.Linear(self.flatten_dim, 2)

    def _dummy_forward(self, input_dim):
        with torch.no_grad():
            x = torch.zeros(1, input_dim)
            x = x.unsqueeze(1)
            x = self.pool1(F.relu(self.conv1(x)))
            x = self.pool2(F.relu(self.conv2(x)))
            x = self.pool3(F.relu(self.conv3(x)))
            self.flatten_dim = x.numel()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        x = x.unsqueeze(1)
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))

        x = x.view(x.size(0), -1)
        return self.fc_out(x)

In [21]:
from sklearn.metrics import accuracy_score, f1_score
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"

def evaluate(model, loader,device):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            logits = model(X)
            pred = torch.argmax(logits, dim=1)

            preds.extend(pred.cpu().numpy())
            labels.extend(y.cpu().numpy())

    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average="macro")
    return acc, macro_f1

In [22]:
def train_model(
    model,
    train_loader,
    dev_loader,
    epochs=10,
    lr=1e-4,
    model_name="model",
    device="cuda"
):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_f1 = 0.0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for X, y in train_loader:
            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)

        dev_acc, dev_f1 = evaluate(model, dev_loader, device)

        print(f"\nEpoch {epoch+1}/{epochs}")
        print(f"Train Loss: {avg_loss:.4f}")
        print(f"Dev Acc   : {dev_acc:.4f}")
        print(f"Dev F1    : {dev_f1:.4f}")

        if dev_f1 > best_f1:
            best_f1 = dev_f1
            torch.save(model.state_dict(), f"{model_name}.pt")
            print(" Best model saved")

    print(f"\n Best Dev Macro-F1 ({model_name}): {best_f1:.4f}")

In [23]:
from torch.utils.data import DataLoader, TensorDataset

def make_loaders(train_emb, dev_emb, test_emb, y_train, y_dev, y_test, batch_size=32):

    train_ds = TensorDataset(
        torch.tensor(train_emb, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.long)
    )

    dev_ds = TensorDataset(
        torch.tensor(dev_emb, dtype=torch.float32),
        torch.tensor(y_dev, dtype=torch.long)
    )

    test_ds = TensorDataset(
        torch.tensor(test_emb, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.long)
    )

    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True),
        DataLoader(dev_ds, batch_size=batch_size),
        DataLoader(test_ds, batch_size=batch_size)
    )

In [32]:
y_train = train["label"].astype(int).tolist()
y_dev   = dev["label"].astype(int).tolist()
y_test  = test["label"].astype(int).tolist()


In [33]:
#mpnet-cnn
train_loader_mpnet, dev_loader_mpnet, test_loader_mpnet = make_loaders(
    train_mpnet, dev_mpnet, test_mpnet,
    y_train, y_dev, y_test
)

model_mpnet = HopeSpeechCNN(input_dim=768).to(device)

train_model(
    model_mpnet,
    train_loader_mpnet,
    dev_loader_mpnet,
    epochs=10,
    lr=1e-4,
    model_name="cnn_mpnet_tamil",
    device=device
)


Epoch 1/10
Train Loss: 0.6857
Dev Acc   : 0.5690
Dev F1    : 0.3626
 Best model saved

Epoch 2/10
Train Loss: 0.6562
Dev Acc   : 0.6231
Dev F1    : 0.5891
 Best model saved

Epoch 3/10
Train Loss: 0.6382
Dev Acc   : 0.6328
Dev F1    : 0.6146
 Best model saved

Epoch 4/10
Train Loss: 0.6283
Dev Acc   : 0.6306
Dev F1    : 0.6158
 Best model saved

Epoch 5/10
Train Loss: 0.6198
Dev Acc   : 0.6323
Dev F1    : 0.6084

Epoch 6/10
Train Loss: 0.6084
Dev Acc   : 0.6420
Dev F1    : 0.6063

Epoch 7/10
Train Loss: 0.5931
Dev Acc   : 0.6351
Dev F1    : 0.6229
 Best model saved

Epoch 8/10
Train Loss: 0.5725
Dev Acc   : 0.6266
Dev F1    : 0.6114

Epoch 9/10
Train Loss: 0.5421
Dev Acc   : 0.6288
Dev F1    : 0.6064

Epoch 10/10
Train Loss: 0.5094
Dev Acc   : 0.6186
Dev F1    : 0.5991

 Best Dev Macro-F1 (cnn_mpnet_tamil): 0.6229


In [34]:
# Create DataLoaders for IndicBERT embeddings
train_loader_indic, dev_loader_indic, test_loader_indic = make_loaders(
    train_indic_emb, dev_indic_emb, test_indic_emb,
    y_train, y_dev, y_test
)

# Initialize model
model_indic = HopeSpeechCNN(input_dim=768).to(device)

# Train model
train_model(
    model_indic,
    train_loader_indic,
    dev_loader_indic,
    epochs=10,
    lr=1e-4,
    model_name="cnn_indicbert_tamil",
    device=device
)


Epoch 1/10
Train Loss: 0.6870
Dev Acc   : 0.5690
Dev F1    : 0.3626
 Best model saved

Epoch 2/10
Train Loss: 0.6769
Dev Acc   : 0.5975
Dev F1    : 0.5594
 Best model saved

Epoch 3/10
Train Loss: 0.6602
Dev Acc   : 0.6106
Dev F1    : 0.5894
 Best model saved

Epoch 4/10
Train Loss: 0.6531
Dev Acc   : 0.6311
Dev F1    : 0.5966
 Best model saved

Epoch 5/10
Train Loss: 0.6475
Dev Acc   : 0.6226
Dev F1    : 0.5889

Epoch 6/10
Train Loss: 0.6448
Dev Acc   : 0.6334
Dev F1    : 0.5985
 Best model saved

Epoch 7/10
Train Loss: 0.6407
Dev Acc   : 0.6135
Dev F1    : 0.5980

Epoch 8/10
Train Loss: 0.6372
Dev Acc   : 0.6163
Dev F1    : 0.6039
 Best model saved

Epoch 9/10
Train Loss: 0.6332
Dev Acc   : 0.6192
Dev F1    : 0.6020

Epoch 10/10
Train Loss: 0.6306
Dev Acc   : 0.6431
Dev F1    : 0.6157
 Best model saved

 Best Dev Macro-F1 (cnn_indicbert_tamil): 0.6157


In [35]:
import numpy as np

# Make sure the order of samples matches
X_train_fused = np.concatenate([train_mpnet, train_indic_emb], axis=1)
X_dev_fused   = np.concatenate([dev_mpnet, dev_indic_emb], axis=1)
X_test_fused  = np.concatenate([test_mpnet, test_indic_emb], axis=1)

print("Fused train shape:", X_train_fused.shape)
print("Fused dev shape  :", X_dev_fused.shape)
print("Fused test shape :", X_test_fused.shape)


Fused train shape: (14186, 1536)
Fused dev shape  : (1754, 1536)
Fused test shape : (1760, 1536)


In [36]:
# Create loaders for fused embeddings
train_loader_fused, dev_loader_fused, test_loader_fused = make_loaders(
    X_train_fused, X_dev_fused, X_test_fused,
    y_train, y_dev, y_test
)
input_dim = X_train_fused.shape[1]  # 768 + 768 = 1536

model_fused = HopeSpeechCNN(input_dim=input_dim).to(device)
train_model(
    model_fused,
    train_loader_fused,
    dev_loader_fused,
    epochs=10,
    lr=1e-4,
    model_name="cnn_fused_mpnet_indic_tamil",
    device=device
)


Epoch 1/10
Train Loss: 0.6833
Dev Acc   : 0.6083
Dev F1    : 0.4679
 Best model saved

Epoch 2/10
Train Loss: 0.6501
Dev Acc   : 0.6203
Dev F1    : 0.6032
 Best model saved

Epoch 3/10
Train Loss: 0.6347
Dev Acc   : 0.6306
Dev F1    : 0.6045
 Best model saved

Epoch 4/10
Train Loss: 0.6258
Dev Acc   : 0.6385
Dev F1    : 0.6057
 Best model saved

Epoch 5/10
Train Loss: 0.6190
Dev Acc   : 0.6448
Dev F1    : 0.5958

Epoch 6/10
Train Loss: 0.6102
Dev Acc   : 0.6442
Dev F1    : 0.6160
 Best model saved

Epoch 7/10
Train Loss: 0.6013
Dev Acc   : 0.6477
Dev F1    : 0.6077

Epoch 8/10
Train Loss: 0.5898
Dev Acc   : 0.6220
Dev F1    : 0.6089

Epoch 9/10
Train Loss: 0.5741
Dev Acc   : 0.6300
Dev F1    : 0.6240
 Best model saved

Epoch 10/10
Train Loss: 0.5508
Dev Acc   : 0.6243
Dev F1    : 0.6197

 Best Dev Macro-F1 (cnn_fused_mpnet_indic_tamil): 0.6240


In [37]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)
import torch
import numpy as np

def evaluate_on_test(model, test_loader, device):
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    f1  = f1_score(all_labels, all_preds, average="macro")
    prec = precision_score(all_labels, all_preds, average="macro")
    rec  = recall_score(all_labels, all_preds, average="macro")

    print("Accuracy      :", round(acc, 4))
    print("Macro-F1      :", round(f1, 4))
    print("Macro-Precision:", round(prec, 4))
    print("Macro-Recall  :", round(rec, 4))
    print("\nClassification Report:\n")
    print(classification_report(all_labels, all_preds, digits=4))

    return acc, f1, prec, rec

In [38]:
# MPNet TEST EVALUATION

model_mpnet = HopeSpeechCNN(input_dim=768).to(device)
model_mpnet.load_state_dict(torch.load("cnn_mpnet_tamil.pt"))
model_mpnet.to(device)

evaluate_on_test(model_mpnet, test_loader_mpnet, device)

Accuracy      : 0.6295
Macro-F1      : 0.6206
Macro-Precision: 0.6277
Macro-Recall  : 0.6215

Classification Report:

              precision    recall  f1-score   support

           0     0.6356    0.7283    0.6788       946
           1     0.6198    0.5147    0.5624       814

    accuracy                         0.6295      1760
   macro avg     0.6277    0.6215    0.6206      1760
weighted avg     0.6283    0.6295    0.6250      1760



(0.6295454545454545, 0.620616920686349, 0.6277156706478307, 0.6215359122335866)

In [39]:
# INDICBERT TEST EVALUATION

model_indic = HopeSpeechCNN(input_dim=768).to(device)
model_indic.load_state_dict(torch.load("cnn_indicbert_tamil.pt"))
model_indic.to(device)

evaluate_on_test(model_indic, test_loader_indic, device)

Accuracy      : 0.621
Macro-F1      : 0.5983
Macro-Precision: 0.6263
Macro-Recall  : 0.6066

Classification Report:

              precision    recall  f1-score   support

           0     0.6131    0.7992    0.6939       946
           1     0.6395    0.4140    0.5026       814

    accuracy                         0.6210      1760
   macro avg     0.6263    0.6066    0.5983      1760
weighted avg     0.6253    0.6210    0.6054      1760



(0.6210227272727272, 0.598253137620682, 0.6263036884167371, 0.6065796240214845)

In [42]:
# FUSION TEST EVALUATION
model_fusion = HopeSpeechCNN(input_dim=768*2).to(device)
model_fusion.load_state_dict(torch.load("cnn_fused_mpnet_indic_tamil.pt"))

evaluate_on_test(model_fusion, test_loader_fused , device)

Accuracy      : 0.6205
Macro-F1      : 0.6162
Macro-Precision: 0.6175
Macro-Recall  : 0.616

Classification Report:

              precision    recall  f1-score   support

           0     0.6390    0.6755    0.6567       946
           1     0.5961    0.5565    0.5756       814

    accuracy                         0.6205      1760
   macro avg     0.6175    0.6160    0.6162      1760
weighted avg     0.6191    0.6205    0.6192      1760



(0.6204545454545455,
 0.6161676576328337,
 0.6175263157894737,
 0.6159933718073253)

In [43]:
results = {}
def evaluate_and_store(name, model, dev_loader, test_loader, device):
    from sklearn.metrics import f1_score
    import torch

    def eval_loader(loader):
        model.eval()
        preds, labels = [], []

        with torch.no_grad():
            for x, y in loader:
                x = x.to(device)
                y = y.to(device)

                logits = model(x)
                preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
                labels.extend(y.cpu().numpy())

        return f1_score(labels, preds, average="macro")

    dev_f1  = eval_loader(dev_loader)
    test_f1 = eval_loader(test_loader)

    results[name] = {
        "Dev Macro-F1": round(dev_f1, 4),
        "Test Macro-F1": round(test_f1, 4)
    }

    print(name)
    print(" Dev Macro-F1 :", round(dev_f1, 4))
    print(" Test Macro-F1:", round(test_f1, 4))
    print("-" * 40)

In [45]:
model_mpnet = HopeSpeechCNN(input_dim=768).to(device)
model_mpnet.load_state_dict(torch.load("cnn_mpnet_tamil.pt"))

evaluate_and_store(
    "MPNet-CNN",
    model_mpnet,
    dev_loader_mpnet,
    test_loader_mpnet,
    device
)

MPNet-CNN
 Dev Macro-F1 : 0.6229
 Test Macro-F1: 0.6206
----------------------------------------


In [46]:
model_indic = HopeSpeechCNN(input_dim=768).to(device)
model_indic.load_state_dict(torch.load("cnn_indicbert_tamil.pt"))

evaluate_and_store(
    "IndicBERT-CNN",
    model_indic,
    dev_loader_indic,
    test_loader_indic,
    device
)

IndicBERT-CNN
 Dev Macro-F1 : 0.6157
 Test Macro-F1: 0.5983
----------------------------------------


In [49]:
model_indic = HopeSpeechCNN(input_dim=768*2).to(device)
model_indic.load_state_dict(torch.load("/kaggle/working/cnn_fused_mpnet_indic_tamil.pt"))

evaluate_and_store(
    "FUSION-CNN",
    model_fusion,
    dev_loader_fused,
    test_loader_fused,
    device
)

FUSION-CNN
 Dev Macro-F1 : 0.624
 Test Macro-F1: 0.6162
----------------------------------------


In [52]:
import pandas as pd

df_results = pd.DataFrame(results).T
df_results

,Dev Macro-F1,Test Macro-F1
MPNet-CNN,0.6229,0.6206
IndicBERT-CNN,0.6157,0.5983
FUSION-CNN,0.6240,0.6162
